In [21]:
cd /Users/edisonlee/Documents/weishi/young/analysis

/Users/edisonlee/Documents/weishi/young/analysis


In [22]:
import math
import numpy as np
import pandas as pd
import cufflinks as cf
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline

In [23]:
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 500)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
cf.go_offline()

In [24]:
cd data/

/Users/edisonlee/Documents/weishi/young/analysis/data


In [26]:
consumeDetailDf = pd.read_csv('热点视频消费数据.csv', header=0, 
                names=[
                '视频id', '性别', '城市等级', '历史发布频次', '大盘播放VV', '年轻人播放VV', '年轻人播放VV占比',
                '大盘完播率', '年轻人完播率', '大盘互动率', '年轻人互动率',
                '大盘单vv时长', '年轻人单vv时长', '大盘播放完成度', '年轻人播放完成度',
                '大盘3s快滑率', '年轻人3s快滑率', '大盘5s快滑率', '年轻人5s快滑率'])
consumeDetailDf = consumeDetailDf.fillna(0)
consumeDetailDf = consumeDetailDf.replace(np.nan, 0)
consumeDetailDf = consumeDetailDf.astype({
                '视频id':str, '性别':str, '城市等级':str, '历史发布频次':str, 
                '大盘播放VV':pd.Int64Dtype(), '年轻人播放VV':pd.Int64Dtype(), '年轻人播放VV占比':float,
                '大盘完播率':float, '年轻人完播率':float, '大盘互动率':float, '年轻人互动率':float,
                '大盘单vv时长':float, '年轻人单vv时长':float, '大盘播放完成度':float, '年轻人播放完成度':float,
                '大盘3s快滑率':float, '年轻人3s快滑率':float, '大盘5s快滑率':float, '年轻人5s快滑率':float})
consumeDetailDf['大盘3s快滑率'] = round(consumeDetailDf['大盘3s快滑率']/100,2)
consumeDetailDf['大盘5s快滑率'] = round(consumeDetailDf['大盘5s快滑率']/100,2)
consumeDetailDf

,视频id,性别,城市等级,历史发布频次,大盘播放VV,年轻人播放VV,年轻人播放VV占比,大盘完播率,年轻人完播率,大盘互动率,年轻人互动率,大盘单vv时长,年轻人单vv时长,大盘播放完成度,年轻人播放完成度,大盘3s快滑率,年轻人3s快滑率,大盘5s快滑率,年轻人5s快滑率
0,7my88ooOF1KO92piP,女,五线,1-5次发布用户,74,24,0.32,0.08,0.08,0.00,0.00,6.29,4.52,0.33,0.40,0.62,0.54,0.73,0.58
1,7my88ooOF1KO92piP,男,四线,>10次发布用户,23,2,0.09,0.09,0.00,0.00,0.00,2.82,0.99,0.27,0.10,0.78,1.00,0.78,1.00
2,7my88ooOF1KO92piP,男,其他,>10次发布用户,1,1,1.00,0.00,0.00,0.00,0.00,0.89,0.89,0.09,0.09,1.00,1.00,1.00,1.00
3,7my88ooOF1KO92piP,男,二线,1-5次发布用户,165,89,0.54,0.06,0.04,0.02,0.02,3.49,3.40,0.34,0.33,0.68,0.71,0.73,0.75
4,7my88ooOF1KO92piP,男,四线,6-10次发布用户,12,1,0.08,0.08,0.00,0.00,0.00,4.53,1.19,0.35,0.12,0.58,1.00,0.67,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6050,6R4836WBo1KNY0Z4r,女,五线,6-10次发布用户,10,2,0.20,0.20,0.50,0.00,0.00,5.06,6.97,0.42,0.60,0.40,0.50,0.70,0.50
6051,6R4836WBo1KNY0Z4r,女,三线,0次发布用户,8413,5029,0.60,0.13,0.12,0.00,0.00,7.28,6.37,0.39,0.37,0.50,0.53,0.65,0.67
6052,6R4836WBo1KNY0Z4r,未知,新一线,0次发布用户,42,27,0.64,0.10,0.11,0.00,0.00,4.58,4.80,0.39,0.41,0.57,0.52,0.64,0.59
6053,6R4836WBo1KNY0Z4r,女,一线,1-5次发布用户,63,24,0.38,0.08,0.04,0.00,0.00,5.01,4.20,0.32,0.33,0.62,0.54,0.75,0.67


In [28]:
consumeStatDf = consumeDetailDf.groupby('视频id').agg({
    '大盘播放VV':'sum', '年轻人播放VV':'sum', '年轻人播放VV占比':['mean','median'], 
    '大盘完播率':['mean','median'], '年轻人完播率':['mean','median'], '大盘互动率':['mean','median'], '年轻人互动率':['mean','median'], 
    '大盘单vv时长':['mean','median'], '年轻人单vv时长':['mean','median'], '大盘播放完成度':['mean','median'], '年轻人播放完成度':['mean','median'], 
    '大盘3s快滑率':['mean','median'], '年轻人3s快滑率':['mean','median'], '大盘5s快滑率':['mean','median'], '年轻人5s快滑率':['mean','median']
})
consumeStatDf.columns = ['_'.join(col) for col in consumeStatDf.columns]
consumeStatDf.reset_index(inplace=True)
consumeStatDf.head(5)

,视频id,大盘播放VV_sum,年轻人播放VV_sum,年轻人播放VV占比_mean,年轻人播放VV占比_median,大盘完播率_mean,大盘完播率_median,年轻人完播率_mean,年轻人完播率_median,大盘互动率_mean,大盘互动率_median,年轻人互动率_mean,年轻人互动率_median,大盘单vv时长_mean,大盘单vv时长_median,年轻人单vv时长_mean,年轻人单vv时长_median,大盘播放完成度_mean,大盘播放完成度_median,年轻人播放完成度_mean,年轻人播放完成度_median,大盘3s快滑率_mean,大盘3s快滑率_median,年轻人3s快滑率_mean,年轻人3s快滑率_median,大盘5s快滑率_mean,大盘5s快滑率_median,年轻人5s快滑率_mean,年轻人5s快滑率_median
0,6R4836WBo1KNA4Xj4,171446,111858,0.36,0.33,0.11,0.12,0.10,0.09,0.01,0.00,0.01,0.00,5.21,5.41,4.75,4.75,0.38,0.42,0.39,0.43,0.58,0.51,0.54,0.50,0.70,0.64,0.66,0.62
1,6R4836WBo1KNY0Z4r,87485,56679,0.36,0.33,0.11,0.11,0.08,0.07,0.01,0.00,0.00,0.00,6.18,5.70,4.99,4.59,0.36,0.36,0.33,0.33,0.55,0.55,0.58,0.56,0.70,0.70,0.74,0.72
2,6R4836WBo1KNY4HLw,95669,62203,0.36,0.35,0.09,0.09,0.09,0.07,0.00,0.00,0.00,0.00,5.81,5.45,5.65,5.25,0.34,0.34,0.31,0.33,0.54,0.52,0.58,0.54,0.70,0.69,0.75,0.73
3,6XfIL4q5R1KNEMn8X,326098,214394,0.40,0.44,0.20,0.22,0.17,0.19,0.01,0.00,0.01,0.00,6.05,6.09,5.32,5.49,0.58,0.59,0.56,0.59,0.41,0.40,0.42,0.40,0.56,0.55,0.57,0.54
4,6XfIL4q5R1KP07BTZ,151866,99630,0.34,0.27,0.16,0.17,0.15,0.15,0.00,0.00,0.00,0.00,5.97,5.82,5.66,5.20,0.46,0.48,0.43,0.45,0.49,0.46,0.53,0.49,0.65,0.63,0.69,0.66


In [61]:
feedAttrDf = pd.read_excel('年轻人冷启投放标签.xlsx', 
            header=0, 
            names=[
                '填写人', '微视账号昵称', '视频id',
                '玩法分类', '玩法名称', '视频时长', '使用素材数量', '视频所在品类', '使用素材类型',
                '是否使用贴纸', '是否使用特效', '素材使用主题', '预期评级',
                '是否蹭热点', '是否带话题', '是否绑定挑战赛', '是否文案引导'],
            dtype={
                '填写人':str, '微视账号昵称':str, '视频id':str,
                '玩法分类':str, '视频时长':float, '使用素材数量':str, '视频所在品类':str, '使用素材类型':str,
                '是否使用贴纸':str, '是否使用特效':str, '素材使用主题':str,
                '是否蹭热点':str, '是否带话题':str, '是否绑定挑战赛':str, '是否文案引导':str
            })
feedAttrDf['视频时长'] = pd.cut(feedAttrDf['视频时长'], [0,8,12,16,20,100], labels=["(0,8]","(8,12]","(12,16]","(16,20]","20s以上"])
# feedAttrDf.fillna(0).head(10)
feedAttrDf.head(8)

,填写人,微视账号昵称,视频id,玩法分类,玩法名称,视频时长,使用素材数量,视频所在品类,使用素材类型,是否使用贴纸,是否使用特效,素材使用主题,预期评级,是否蹭热点,是否带话题,是否绑定挑战赛,是否文案引导
0,v_lilyyyxie,探长的私家日记,6XfIL4q5R1KNEMn8X,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,有
1,v_lilyyyxie,天乐,7bbDTvnKj1KOG5zp3,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,B,无,有,无,有
2,v_lilyyyxie,小七菌,7f2kIDSCa1KNAntKM,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,有,有,无,有
3,v_lilyyyxie,彭豆豆,7dDtw3z2e1KND34Yu,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,C,无,有,无,无
4,v_lilyyyxie,贾鑫,787S1I3KY1KOmyDAT,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,萌娃,A,无,有,无,有
5,v_lilyyyxie,卿酒酒,7jXQZOpVi1KNLtqgN,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,有,有,无,有
6,v_lilyyyxie,粉衣少年,7iEORw1251KOnJH6O,模板,变装派对,"(8,12]",1-2张,颜值,照片,有,有,性感女性,A,无,有,无,有
7,v_lilyyyxie,慢慢要早睡,7jowa21U01KOnHDDt,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,有,有,无,有


In [62]:
# Merge feedAttr with consumeDetailData, then sort
feedAttrConsumeDetailDf = pd.merge(feedAttrDf, consumeDetailDf, on='视频id', how='left')
feedAttrConsumeDetailDf.sort_values(by=['视频id', '大盘播放VV'], ascending=[True, False], inplace=True)
feedAttrConsumeDetailDf

,填写人,微视账号昵称,视频id,玩法分类,玩法名称,视频时长,使用素材数量,视频所在品类,使用素材类型,是否使用贴纸,是否使用特效,素材使用主题,预期评级,是否蹭热点,是否带话题,是否绑定挑战赛,是否文案引导,性别,城市等级,历史发布频次,大盘播放VV,年轻人播放VV,年轻人播放VV占比,大盘完播率,年轻人完播率,大盘互动率,年轻人互动率,大盘单vv时长,年轻人单vv时长,大盘播放完成度,年轻人播放完成度,大盘3s快滑率,年轻人3s快滑率,大盘5s快滑率,年轻人5s快滑率
1828,v_lrenechen,程宝Angela,6R4836WBo1KNA4Xj4,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,无,男,三线,0次发布用户,18716,13371,0.71,0.15,0.14,0.00,0.00,6.26,6.30,0.44,0.43,0.50,0.51,0.63,0.63
1854,v_lrenechen,程宝Angela,6R4836WBo1KNA4Xj4,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,无,男,四线,0次发布用户,16756,12359,0.74,0.16,0.15,0.00,0.00,6.47,6.38,0.44,0.44,0.49,0.50,0.62,0.62
1840,v_lrenechen,程宝Angela,6R4836WBo1KNA4Xj4,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,无,女,三线,0次发布用户,16172,9597,0.59,0.17,0.16,0.00,0.00,7.29,7.20,0.45,0.45,0.48,0.48,0.61,0.61
1813,v_lrenechen,程宝Angela,6R4836WBo1KNA4Xj4,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,无,女,四线,0次发布用户,15452,9568,0.62,0.18,0.17,0.00,0.00,7.21,6.93,0.46,0.46,0.46,0.47,0.60,0.60
1846,v_lrenechen,程宝Angela,6R4836WBo1KNA4Xj4,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,无,男,二线,0次发布用户,15321,11443,0.75,0.14,0.14,0.00,0.01,6.02,5.94,0.44,0.44,0.50,0.50,0.63,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,v_yanppeng,猪头,7my88ooOF1KO92piP,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,有,无,无,无,未知,三线,1-5次发布用户,1,1,1.00,0.00,0.00,0.00,0.00,2.83,2.83,0.29,0.29,1.00,1.00,1.00,1.00
2351,v_yanppeng,猪头,7my88ooOF1KO92piP,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,有,无,无,无,未知,其他,6-10次发布用户,1,0,0.00,0.00,nan,0.00,nan,1.12,nan,0.11,nan,1.00,nan,1.00,nan
2368,v_yanppeng,猪头,7my88ooOF1KO92piP,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,有,无,无,无,男,五线,>10次发布用户,1,0,0.00,0.00,nan,0.00,nan,1.95,nan,0.20,nan,1.00,nan,1.00,nan
2390,v_yanppeng,猪头,7my88ooOF1KO92piP,模板,变装派对,"(8,12]",1-2张,颜值,照片,无,无,清纯妹子,A,有,无,无,无,男,一线,>10次发布用户,1,1,1.00,0.00,0.00,0.00,0.00,2.54,2.54,0.26,0.26,1.00,1.00,1.00,1.00


In [63]:
# Merge feedAttr with consumeStatData, then sort
feedAttrConsumeStatDf = pd.merge(feedAttrDf, consumeStatDf, on='视频id', how='left')
feedAttrConsumeStatDf.sort_values(by=['大盘播放VV_sum'], ascending=False, inplace=True)
feedAttrConsumeStatDf.head(5)

,填写人,微视账号昵称,视频id,玩法分类,玩法名称,视频时长,使用素材数量,视频所在品类,使用素材类型,是否使用贴纸,是否使用特效,素材使用主题,预期评级,是否蹭热点,是否带话题,是否绑定挑战赛,是否文案引导,大盘播放VV_sum,年轻人播放VV_sum,年轻人播放VV占比_mean,年轻人播放VV占比_median,大盘完播率_mean,大盘完播率_median,年轻人完播率_mean,年轻人完播率_median,大盘互动率_mean,大盘互动率_median,年轻人互动率_mean,年轻人互动率_median,大盘单vv时长_mean,大盘单vv时长_median,年轻人单vv时长_mean,年轻人单vv时长_median,大盘播放完成度_mean,大盘播放完成度_median,年轻人播放完成度_mean,年轻人播放完成度_median,大盘3s快滑率_mean,大盘3s快滑率_median,年轻人3s快滑率_mean,年轻人3s快滑率_median,大盘5s快滑率_mean,大盘5s快滑率_median,年轻人5s快滑率_mean,年轻人5s快滑率_median
4,v_lilyyyxie,贾鑫,787S1I3KY1KOmyDAT,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,萌娃,A,无,有,无,有,362004,235433,0.38,0.43,0.28,0.28,0.24,0.24,0.00,0.00,0.00,0.00,6.51,6.35,5.71,5.68,0.63,0.61,0.59,0.61,0.38,0.40,0.43,0.41,0.51,0.53,0.54,0.53
0,v_lilyyyxie,探长的私家日记,6XfIL4q5R1KNEMn8X,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,无,有,无,有,326098,214394,0.40,0.44,0.20,0.22,0.17,0.19,0.01,0.00,0.01,0.00,6.05,6.09,5.32,5.49,0.58,0.59,0.56,0.59,0.41,0.40,0.42,0.40,0.56,0.55,0.57,0.54
27,v_lrenechen,黄金屋&颜如玉,7eRuPfWvz1KNAI6G4,模板,变装派对,"(8,12]",1-2张,亲子,照片,有,无,萌娃,B,无,有,无,无,317702,208805,0.37,0.33,0.24,0.23,0.22,0.19,0.01,0.00,0.01,0.00,7.65,7.61,7.30,6.80,0.57,0.56,0.56,0.56,0.32,0.33,0.34,0.33,0.46,0.46,0.47,0.46
20,v_lilyyyxie,喜喜和恺恺,7cl3wXSYn1KO0IUYl,模板,变装派对,"(0,8]",1-2张,亲子,照片,无,无,萌娃,A,无,有,无,有,312739,201895,0.37,0.38,0.22,0.23,0.20,0.20,0.00,0.00,0.01,0.00,6.10,6.00,6.31,5.19,0.54,0.56,0.53,0.55,0.47,0.44,0.47,0.45,0.60,0.57,0.60,0.58
5,v_lilyyyxie,卿酒酒,7jXQZOpVi1KNLtqgN,模板,变装派对,"(0,8]",1-2张,颜值,照片,无,无,清纯妹子,A,有,有,无,有,312436,209873,0.40,0.41,0.21,0.21,0.16,0.14,0.01,0.00,0.01,0.00,6.43,6.15,5.99,5.23,0.54,0.55,0.50,0.54,0.45,0.43,0.49,0.45,0.57,0.56,0.59,0.56


In [10]:
#视频时长维度拆解

In [83]:
duration_feedAttrConsumeStatDf = feedAttrConsumeStatDf.groupby(['玩法名称','视频时长']).agg({
    '视频id':'count',
    '大盘播放VV_sum':'sum', '年轻人播放VV_sum':'sum',
    '大盘完播率_mean':'mean', '大盘完播率_median':'median',
    '年轻人完播率_mean':'mean', '年轻人完播率_median':'median',
    '大盘互动率_mean':'mean', '大盘互动率_median':'median',
    '年轻人互动率_mean':'mean', '年轻人互动率_median':'median',
    '大盘单vv时长_mean':'mean', '大盘单vv时长_median':'median',
    '年轻人单vv时长_mean':'mean', '年轻人单vv时长_median':'median',
    '大盘播放完成度_mean':'mean', '大盘播放完成度_median':'median',
    '年轻人播放完成度_mean':'mean', '年轻人播放完成度_median':'median',
    '大盘3s快滑率_mean':'mean', '大盘3s快滑率_median':'median',
    '年轻人3s快滑率_mean':'mean', '年轻人3s快滑率_median':'median',
    '大盘5s快滑率_mean':'mean', '大盘5s快滑率_median':'median',
    '年轻人5s快滑率_mean':'mean', '年轻人5s快滑率_median':'median'
}).reset_index()
duration_feedAttrConsumeStatDf.rename(columns={'视频id':'视频数量'}, inplace=True)
duration_feedAttrConsumeStatDf['大盘条均播放VV'] = duration_feedAttrConsumeStatDf['大盘播放VV_sum'] / duration_feedAttrConsumeStatDf['视频数量']
duration_feedAttrConsumeStatDf['年轻人条均播放VV'] = duration_feedAttrConsumeStatDf['年轻人播放VV_sum'] / duration_feedAttrConsumeStatDf['视频数量']

duration_feedAttrConsumeStatDfTidy = pd.melt(duration_feedAttrConsumeStatDf, id_vars=['玩法名称','视频时长'])
#统计方式
duration_feedAttrConsumeStatDfTidy['统计方式'] = pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('mean|条均'), '平均', 
                                                pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('median'), '中位数', 
                                                pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('sum'), '累加', 'other')))
#人群
duration_feedAttrConsumeStatDfTidy['人群'] = pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('大盘'), '大盘', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('年轻人'), '年轻人', 'other'))

#指标
duration_feedAttrConsumeStatDfTidy['指标'] = pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('完播率'), '完播率', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('互动率'), '互动率', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('单vv时长'), '单vv时长', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('播放完成度'), '播放完成度', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('3s快滑率'), '3s快滑率', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('5s快滑率'), '5s快滑率', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('条均播放VV'), '条均播放VV', 
                                            pd.np.where(duration_feedAttrConsumeStatDfTidy.variable.str.contains('播放VV'), '播放VV', 'other'))))))))

duration_feedAttrConsumeStatDfTidy.fillna(value={'value':0}, inplace=True)
duration_feedAttrConsumeStatDfTidy.head(10)

,玩法名称,视频时长,variable,value,统计方式,人群,指标
0,变装派对,"(0,8]",视频数量,15.00,other,other,other
1,变装派对,"(8,12]",视频数量,24.00,other,other,other
2,变装派对,"(12,16]",视频数量,0.00,other,other,other
3,变装派对,"(16,20]",视频数量,0.00,other,other,other
4,变装派对,20s以上,视频数量,0.00,other,other,other
5,牛奶颂,"(0,8]",视频数量,12.00,other,other,other
6,牛奶颂,"(8,12]",视频数量,26.00,other,other,other
7,牛奶颂,"(12,16]",视频数量,4.00,other,other,other
8,牛奶颂,"(16,20]",视频数量,1.00,other,other,other
9,牛奶颂,20s以上,视频数量,5.00,other,other,other


In [88]:
duration_feedAttrConsumeStatDfTidy.query("variable.str.contains('播放VV') & ~(variable.str.contains('条均'))").head(50)

,玩法名称,视频时长,variable,value,统计方式,人群,指标
10,变装派对,"(0,8]",大盘播放VV_sum,3805891.00,累加,大盘,播放VV
11,变装派对,"(8,12]",大盘播放VV_sum,3807969.00,累加,大盘,播放VV
12,变装派对,"(12,16]",大盘播放VV_sum,0.00,累加,大盘,播放VV
13,变装派对,"(16,20]",大盘播放VV_sum,0.00,累加,大盘,播放VV
14,变装派对,20s以上,大盘播放VV_sum,0.00,累加,大盘,播放VV
15,牛奶颂,"(0,8]",大盘播放VV_sum,1879873.00,累加,大盘,播放VV
16,牛奶颂,"(8,12]",大盘播放VV_sum,2898338.00,累加,大盘,播放VV
17,牛奶颂,"(12,16]",大盘播放VV_sum,407548.00,累加,大盘,播放VV
18,牛奶颂,"(16,20]",大盘播放VV_sum,71675.00,累加,大盘,播放VV
19,牛奶颂,20s以上,大盘播放VV_sum,293720.00,累加,大盘,播放VV


In [84]:
#制图

#其它消费指标
duration_feedAttrConsumeStatFig = px.bar(
                                duration_feedAttrConsumeStatDfTidy.query("玩法名称.str.contains('牛奶颂') & (~(variable.str.contains('播放VV') or variable.str.contains('单vv时长') or variable.str.contains('互动率') or variable.str.contains('视频数量')))"), 
                                title='视频时长_消费指标对比',
                                x='视频时长', y='value', color='人群', facet_row='统计方式', facet_col='指标', facet_row_spacing=0.05, facet_col_spacing=0.03,
                                labels={"value": "统计值"},
                                barmode='group', template='simple_white', color_discrete_sequence=px.colors.sequential.Blugrn)
duration_feedAttrConsumeStatFig.update_traces(texttemplate='%{value:%.2f}', textposition='outside')
duration_feedAttrConsumeStatFig.update_yaxes(showgrid=True) # the y-axis
duration_feedAttrConsumeStatFig.update_layout(width=1200, height =800, 
                                title=dict(x=0.5, xref='paper', font=dict(size=16)),
                                font_family="PingFangSC-Light",
                                margin=dict(l=30, r=30, b=20, t=100),
                                bargap=0.3, # gap between bars of adjacent location coordinates.
                                bargroupgap=0, # gap between bars of the same location coordinate.
                                hovermode="closest")
duration_feedAttrConsumeStatFig.show()

In [96]:
feedAttrConsumeStatDf.query("素材使用主题.str.contains('旅行')")[{'视频id','大盘播放VV_sum'}]

,视频id,大盘播放VV_sum
62,787S1I3KY1KOZCtZv,168449
